In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import numpy as np
import random
from multiprocessing import Pool

# Lectura de datos
data = np.loadtxt('/content/drive/MyDrive/MÁSTER/IC/P2/data/tai256c.dat', skiprows=1)
flow = np.int32(data[:256])
distances = np.int32(data[256:])

seed = 2024
random.seed(seed)
np.random.seed(seed)

In [3]:
# Generación de la población inicial
def generate_population(population_size, n=256):
    return np.array([np.random.permutation(np.arange(n)) for _ in range(population_size)])

def fitness_pop(population, flow, distances):
    return np.sum(flow[np.newaxis, :, :] * distances[population[:, :, np.newaxis], population[:, np.newaxis, :]],
                  axis=(1, 2))

In [4]:
# Función de selección por torneo
def tournament_selection(population, fitness_vals, tournament_size=5):
    selected = []
    for _ in range(len(population)):
        tournament = random.sample(range(len(population)), tournament_size)
        winner = tournament[np.argmin([fitness_vals[i] for i in tournament])]
        selected.append(population[winner])
    return np.array(selected)

In [5]:
# Operador de cruce por orden (OX)
def ox_crossover(parent1, parent2):
    size = len(parent1)
    start, end = sorted(np.random.choice(range(size), 2, replace=False))
    child = -np.ones(size, dtype=int)
    child[start:end+1] = parent1[start:end+1]
    idx = 0
    for i in range(size):
        if child[i] == -1:
            while parent2[idx] in child:
                idx += 1
            child[i] = parent2[idx]
    return child

In [6]:
# Mutación por intercambio (Swap)
def swap_mutation(individual, mutation_prob=0.05):
    if random.random() < mutation_prob:
        i, j = np.random.choice(range(len(individual)), 2, replace=False)
        individual[i], individual[j] = individual[j], individual[i]
    return individual

In [7]:
def hill_climbing_optimized(individual, flow, distances, max_iterations=100):
    current = individual.copy()
    current_fitness = fitness_pop(np.array([current]), flow, distances)[0]
    for _ in range(max_iterations):
        i, j = np.random.choice(range(len(current)), 2, replace=False)
        neighbor = current.copy()
        neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
        # Cálculo delta:
        delta_fitness = calculate_fitness_delta(current, i, j, flow, distances)
        if delta_fitness < 0: # Para aceptar solo mejoras
            current = neighbor
            current_fitness += delta_fitness
    return current, current_fitness

def calculate_fitness_delta(individual, i, j, flow, distances):
    n = len(individual)
    delta = 0
    for k in range(n):
        if k != i and k != j:
            delta += (flow[i, k] + flow[k, i]) * (distances[individual[j], individual[k]] - distances[individual[i], individual[k]])
            delta += (flow[j, k] + flow[k, j]) * (distances[individual[i], individual[k]] - distances[individual[j], individual[k]])
    return delta

In [8]:
# Algoritmo genético con variante lamarckiana
def lamarckian_genetic_algorithm(flow, distances, population_size=500, generations=200, crossover_prob=0.9, mutation_prob=0.05, tournament_size=5):
    population = generate_population(population_size)

    for generation in range(generations):
        fitness_vals = fitness_pop(population, flow, distances)

        # Aplicar hill climbing a toda la población (herencia directa)
        improved_population = []
        for ind in population:
            improved_ind, improved_fitness = hill_climbing_optimized(ind, flow, distances)
            improved_population.append(improved_ind)
        population = np.array(improved_population)

        # Selección
        fitness_vals = fitness_pop(population, flow, distances)
        selected_population = tournament_selection(population, fitness_vals, tournament_size)

        # Cruzamiento
        offspring = []
        for i in range(0, population_size, 2):
            if random.random() < crossover_prob:
                child1 = ox_crossover(selected_population[i], selected_population[i+1])
                child2 = ox_crossover(selected_population[i+1], selected_population[i])
            else:
                child1, child2 = selected_population[i], selected_population[i+1]
            offspring.append(child1)
            offspring.append(child2)

        # Mutación
        population = np.array([swap_mutation(ind, mutation_prob) for ind in offspring])

        # Imprimir mejor costo
        best_idx = np.argmin(fitness_vals)
        print(f"Generación {generation+1}: Mejor coste = {fitness_vals[best_idx]}")

    fitness_vals = fitness_pop(population, flow, distances)
    best_idx = np.argmin(fitness_vals)
    return population[best_idx], fitness_vals[best_idx]

In [9]:
# Ejecución del algoritmo
best_solution_lamarck, best_cost_lamarck = lamarckian_genetic_algorithm(flow, distances)

print("\nMejor solución encontrada (Lamarckiano):", best_solution_lamarck)
print("Coste asociado a la mejor solución (Lamarckiano):", best_cost_lamarck)

Generación 1: Mejor coste = 48163922
Generación 2: Mejor coste = 47660068
Generación 3: Mejor coste = 47171664
Generación 4: Mejor coste = 46948284
Generación 5: Mejor coste = 46635566
Generación 6: Mejor coste = 46557608
Generación 7: Mejor coste = 46343906
Generación 8: Mejor coste = 46540884
Generación 9: Mejor coste = 46444958
Generación 10: Mejor coste = 46141070
Generación 11: Mejor coste = 46118736
Generación 12: Mejor coste = 45942584
Generación 13: Mejor coste = 45942584
Generación 14: Mejor coste = 45993464
Generación 15: Mejor coste = 45886050
Generación 16: Mejor coste = 45847436
Generación 17: Mejor coste = 45796578
Generación 18: Mejor coste = 45522336
Generación 19: Mejor coste = 45508876
Generación 20: Mejor coste = 45508876
Generación 21: Mejor coste = 45679576
Generación 22: Mejor coste = 45693516
Generación 23: Mejor coste = 45619302
Generación 24: Mejor coste = 45626616
Generación 25: Mejor coste = 45604022
Generación 26: Mejor coste = 45552748
Generación 27: Mejor 